## Example: BAO and $\sigma_8(z)$ forecasts


$\verb|FishLSS|$ requires $\verb|velocileptors|$ to run, which can be installed with: `python3 -m pip install -v git+https://github.com/sfschen/velocileptors`.

This notebook needs to be run from within the `FishLSS` directory.

In [1]:
# import revelant packages
from headers import *
from twoPoint import *
from twoPointNoise import *

In [ ]:
# filename for example survey
bfn = 'example_survey'

-------

##  <span style="color:red"> (0) To speed up the process: </span>

It is significantly faster to set up the forecast (and compute derivatives) with a dedicated computing node rather than from within a jupyter notebook. To do so, run the following from an interactive node (from within the `FishLSS` directory):

```
python example_setup_BAO_sigma8.py example_survey setup &
python example_setup_BAO_sigma8.py example_survey rec &
python example_setup_BAO_sigma8.py example_survey fs &
python example_setup_BAO_sigma8.py example_survey lens &
wait
```

and replace `example survey` with `bfn`. If any of these process run over the time limit, simply rerun them. $\verb|FishLSS|$ will pick up where it left off.

Optional: the number of bins and $f_\text{sky}$ can be passed as arguments when calling `example_setup_BAO_sigma8.py`, should you want to change them from their default values (3, 0.5 respectively for this example).

-------

## (1) Setting up a $\verb|FishLSS|$ forecast

Most of the work has already been done in step <span style="color:red"> (0) </span>. After setting up a forecast, $\verb|FishLSS|$ will create a JSON file that summarizes the forecast assumptions and inputs. We will use this to load the relevant information when building the forecast from within the notebook.

In [3]:
f = open('output/'+bfn+'/summary.json')
summary = json.load(f)

### (1a) The cosmology

A $\verb|FishLSS|$ forecast requires two main ingredients: a fiducial cosmology and an experiment. For the input cosmology, we use a $\verb|CLASS|$ object. 

In [7]:
params = summary['CLASS default parameters']
cosmo = Class() 
cosmo.set(params) 
cosmo.compute() 

### (1b) The experiment

Now we specify the experiment, which is an instance of $\verb|experiment.py|$. At a minimum, we need to specify the redshift range of the survey ($z_\text{min}$ and $z_\text{max}$), the redshift binning, the sky coverage $f_\text{sky}$, the linear bias $b(z)$, and the number density $\bar{n}(z)$. The redshift binning can be specified in two ways: you can either input a `zedges` (numpy array) to specify the edges of the bins, or `nbins` (integer), in which case the redshift bins are assumed to be linearly spaced in $z$. Here we set `nbins = 3` for simplicity, so that we have three redshift bins with $\Delta z=1$.

In [3]:
# load fiducial linear bias/number density from table
zs,bs,ns = np.genfromtxt(bfn + '.txt').T

# assume zs spans the full survey
ze = summary['Edges of redshift bins']
zmin = ze[0]
zmax = ze[-1]

# interpolate
b = interp1d(zs,bs)
n = interp1d(zs,ns)

nbins = len(ze)-1
fsky = summary['fsky']

We can now create the experiment:

In [4]:
exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n)

In addition to the above, one can optionally specify the following in an `experiment` object.

- `b2` (function of $z$): quadratic bias $b_2(z)$ of the tracer, default $b_2 = 8(b-1)/21$
- `sigv` (float): the comoving velocity dispersion for FoG contributions [km/s], default is 100 km/s
- `sigma_z` (float): redshift error $\sigma_z/(1+z)$, assumed to be independent of redshift, default is 0

### (1c) The forecast object

With a cosmology and an experiment in hand, we can now create a forecast. Running the line below will create an `output` directory, as well as a subdirectory for the experiment of interest: `output/example_survey` in this case. After creating these directories, $\verb|FishLSS|$ will calculate the fiducial power spectra ($P_{gg}(\boldsymbol{k},z)$ and $P_\text{recon}(\boldsymbol{k},z)$) at the center of each redshift bin, and store them in `output/example_survey/derivatives/` and `output/example_survey/derivatives_recon` respectively (assuming that the files don't already exist). $\verb|FishLSS|$ will also calculate $C^{\kappa\kappa}_\ell$, $C^{\kappa g_i}_\ell$ and $C^{g_i g_i}_\ell$ for each redshift bin, and save them in `output/example_survey/derivatives_Cl`. 

In [5]:
# By default FishLSS won't recompute the fiducial power spectra (unless overwrite=True),
# instead, FishLSS will load them into memory. (assuming "python example_BAO_sigma8.py setup"
# has been run)
name = summary['Forecast name']
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name=name)

Redshift-space power spectra are computed on a (flattened) $k-\mu$ grid. That is, $P_{gg}(\boldsymbol{k},z)$ is stored as an array of length `forecast.Nk * forecast.Nmu`, with the corresponding values of $k$ and $\mu$ stored in `forecast.k` and `forecast.mu`. 

## (2) BAO forecast

As described in $\S3.6$ of [2106.09713](https://arxiv.org/pdf/2106.09713.pdf), we hold the shape of the fiducial power spectrum fixed in our BAO forecasts. We then find the errors on the two A-P parameters ($\alpha_\perp$, $\alpha_\parallel$) after marginalizing over the linear bias $b$ and 15 broad-band polynomials $\sum_{n=0}^4\sum_{m=0}^2 c_{nm}k^n\mu^{2m}$. We finally intepret the errors on the A-P parameters as the relative errors of $D_A(z)/r_d$ and $H(z)r_d$, where $r_d$ is sound horizon at the drag epoch.

Marginalizing over the polynomial coefficients is trivial to do analytically, so we only need to numerically compute derivatives with respect to $\alpha_\perp,\alpha_\parallel$ and $b$. 

In [6]:
basis = np.array(['alpha_perp','alpha_parallel','b'])

# set recon = True, so that we perform BAO reconstruction when computing the power spectrum
forecast.recon = True

# set the "marginalized parameters", aka the derivatives, to be [alpha's, linear b]
forecast.free_params = basis

In [ ]:
# derivatives should have already been computed using "python example_BAO_sigma8.py example_survey rec"

# can also simply use:
# forecast.compute_derivatives()
# but this is slow in Jupyter

The derivatives have been automatically stored in `output/example_survey/derivatives_recon`. To load these derivatives, simply run:

In [7]:
derivs = forecast.load_derivatives(basis)

Now let's compute the Fisher matrices in each redshift bin using `get_fisher`, which takes the arguments:

- `basis` (np array): basis of the Fisher matrix 
- `globe` (int): let's not worry about this for now, it's value isn't important for computing the Fisher matrix for a single redshift bin
- `derivatives` (np array): load the derivatives from memory, if not specificied $\verb|FishLSS|$ will recalculate them (takes a lot of time!)
- `zbins` (np array): an array of ints specifying which redshift bins to include in the Fisher matrix. In this case we're computing a Fisher matrix for each redshift bin, so we set `zbins = np.array([i])` to get the Fisher matrix for the i'th bin.

In addition the the above you can also specify `kmax` or `kmax_knl` (ratio of $k_\text{max}$ to the non-linear scale at the center of each redshift bin). By default we set `kmax_knl=1` and `kmax=-10`. If `kmax` is set to be a positive number, then the code ignores `kmax_knl` and uses `kmax` instead.

In [8]:
# get the fisher matrices in each of the 3 redshift bins
F = lambda i: forecast.gen_fisher(basis, 100, derivatives=derivs, zbins=np.array([i]))
Fs = [F(i) for i in range(nbins)]
Fs = np.array(Fs)

Since we set `recon = True` when computing the Fisher matrices, $\verb|FishLSS|$ automatically knows to marginalize over the 15 polynomials, so each Fisher matrix will be an $18\times18$ matrix with basis $\{\alpha_\perp,\alpha_\parallel,b,c_{00},\cdots\}$

In [9]:
print(Fs.shape)

(3, 18, 18)


Now lets invert and compute the errors on the A-P parameters.

In [10]:
Finvs = [np.linalg.inv(Fs[i]) for i in range(nbins)]
saperp = [np.sqrt(Finvs[i][0,0]) for i in range(nbins)]
saparr = [np.sqrt(Finvs[i][1,1]) for i in range(nbins)]

From which we get the relative error on $D_A(z)/r_d$ and $H(z)r_d$ in each redshift bin:

In [11]:
print('Relative error on DA/rd:',saperp)
print('Relative error on H*rd:',saparr)

Relative error on DA/rd: [0.003181433656693976, 0.001842152405859197, 0.003276545170228536]
Relative error on H*rd: [0.004565731326440649, 0.0028150515717583935, 0.004831631731011703]


We're done with BAO forecasting, so let's set `recon = False`

In [12]:
forecast.recon = False

## (3) $\sigma_8(z)$ forecast

### (3a) From full-shape data only


Now we'll compute the relative error on $\sigma_8(z)$ in each of the three redshift bins, following the methods described in $\S4.3$ of [2106.09713](https://arxiv.org/pdf/2106.09713.pdf). Here we are going to fix the shape of the linear power spectrum. In this case $\sigma_8^2\propto A_s$ is entirely determined by the primordial amplitude $A_s$. At the level of derivatives this implies $\partial_{\ln \sigma_8(z)} = 2\partial_{\ln A_s}$, so the relative error on $\sigma_8(z)$ is half the relative error on $A_s$. 


First we have to compute the relevant derivatives of the full shape power spectrum. This takes an hour-ish.

In [13]:
basis = np.array(['log(A_s)','N','alpha0','b','b2','bs','N2','N4','alpha2','alpha4'])

forecast.free_params = basis

In [ ]:
# again, these derivatives should have already been computed

# forecast.compute_derivatives()  <- slow version from withing the Jupyter notebook

Let's load the derivatives

In [14]:
derivatives = forecast.load_derivatives(basis)

and compute the Fisher matrices in each bin, which has a 10-dimensional basis $$\{\ln(A_s),N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i),N_2(z_i),N_4(z_i),\alpha_2(z_i),\alpha_4(z_i)\}$$
The order that I chose for the nuisance terms in this basis might seem odd to you, but this will become more clear when combining full shape information with CMB lensing.

In [15]:
F = lambda i: forecast.gen_fisher(basis,100,kmax_knl=1.,derivatives=derivatives, zbins = np.array([i]))
Fs = np.array([F(i) for i in range(nbins)])
Finvs = np.array([np.linalg.inv(F) for F in Fs])

In [16]:
def get_s8_constraint(i): return np.sqrt(Finvs[i][0,0])/2 # divide by two to convert A -> s8
s8_constraints = np.array([get_s8_constraint(i) for i in range(nbins)])

print('Relative constraints on sigma8(z) (fixed shape):',s8_constraints)

Relative constraints on sigma8(z) (fixed shape): [0.01072835 0.00743769 0.01343389]


### (3b) adding CMB lensing $\times$ galaxies 

We first need to compute the relevant derivatives of $(C^{\kappa\kappa}_\ell, C^{\kappa g_i}_\ell, C^{g_ig_i}_\ell)$.

In [17]:
basis_lensing = np.array(['log(A_s)','N','alpha0','b','b2','bs','alphax'])
forecast.free_params = basis_lensing

In [ ]:
# again, these derivatives should have already been computed

# forecast.compute_Cl_derivatives()  <- slow version from withing the Jupyter notebook

Now we'll compute the CMB lensing $\times$ galaxies Fisher matrices in each bin using `gen_lensing_fisher`, which takes the following inputs:

- `basis_lensing` (np array): the basis of the Fisher matrix
- `globe_lensing` (np array): number of global parameters (parameters which don't depend on the redshift bin), in this case 1 ($\ln(A_s)$)
- `ell_min`, `ell_max` (ints): multipoles to include in the Fisher matrix
- `bins` (np array): array of ints to specify which redshift bins to include in the Fisher matrix, default is to include all the redshift bins
- `kk` (boolean): Set False to remove $C^{\kappa\kappa}_\ell$ from the data vector, default is True
- `CMB` (string): Choose from 'Planck', 'SO' or 'S4' to set lensing noise levels 

In our example, the lensing Fisher matrix will have basis $\{\ln(A_s),N_0(z_1),\cdots,N_0(z_n),\alpha_0(z_1),\cdots,\alpha_0(z_n),b(z_1),\cdots\}$, where $z_i$ is the central redshift of the i'th bin. In this example $i=0,1,2$. This is true regardless of the `bins` input. However, if we're only including one redshift bin in our data vector, then our data is obviously insensitive to the nuisance parameters in the other bins. Thus when setting `bins = np.array([i])`, we can remove the $N_0(z_j),\alpha_0(z_j),\cdots$ ($j\neq i$) columns/rows from the Fisher matrix (which are all zero), so that our basis becomes: 
$$\{\ln(A_s),N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i),\alpha_x(z_i)\}$$
Below is a helper function `get_lensing_fishers` which deletes these unnessecary rows. 

In [18]:
def get_lensing_fishers(cast):
   globe_lensing = 1
   # xs is a set of indicies telling me the relevant terms for the i'th redshift bin
   xs = [[0] # log(A_s)
         +
         [int(1+0*nbins+i),int(1+1*nbins+i),int(1+2*nbins+i), # relevant nuisance terms (6 of them)
          int(1+3*nbins+i),int(1+4*nbins+i),int(1+5*nbins+i)]
         for i in range(nbins)]
   Lensing = [forecast.gen_lensing_fisher(basis_lensing,globe_lensing,ell_min=30,ell_max=500,
                                          bins=np.array([i]),kk=False,CMB='S4') for i in range(nbins)]
   Short = [Lensing[i][xs[i]][:,xs[i]] for i in range(nbins)] # remove all off the irrelevant columns/rows
   return np.array(Short)

In [19]:
Fs_lensing = get_lensing_fishers(forecast)

Now we are going to combine the CMB lensing $\times$ galaxies Fishers with the full-shape Fishers.

To combine the two Fishers we can just use the `combine_fishers` function, with `globe = 6`. 

Definition of `globe` (int): Let $\{a_1,\cdots,a_n,b_1,\cdots,b_m\}$ be the basis of the first Fisher matrix and $\{a_1,\cdots,a_n,c_1,\cdots,c_l\}$ be the basis of the second Fisher matrix, where I'm assuming that the $b_i$'s and $c_i$'s are distinct parameters (such as two different sets of nuisance parameters), then `globe = n`. That is, `globe` counts the number of "global" parameters that are common to the Fisher matrices. In this case we are combining a $10\times10$ Fisher (full-shape) with a $7\times7$ Fisher (lensing), with the first six parameters being identical $\{\ln(A_s),N_0(z_i),\alpha_0(z_i),b(z_i),b_2(z_i),b_s(z_i)
\}$. So for this case we set `globe = 6`. <span style="color:red">This function assumes that the Fishers have the same basis up to the globe'th entry.</span> The basis of the combined fisher is $\{a_1,\cdots,a_n,b_1,\cdots,b_m,c_1,\cdots,c_l\}$.

In [20]:
Fs_combined = []
for i in range(nbins): 
    fishers = [Fs[i],Fs_lensing[i]] # full-shape, CMB x galaxies
    globe = 6
    combined = forecast.combine_fishers(fishers,globe)
    Fs_combined.append(combined)
Fs_combined = np.array(Fs_combined)
Finvs_combined_fixed = np.array([np.linalg.inv(Fs_combined[i]) for i in range(nbins)])

In [21]:
def get_s8_constraint(i): return np.sqrt(Finvs_combined_fixed[i][0,0])/2 # divide by two to convert A -> s8
s8_constraints_combined = np.array([get_s8_constraint(i) for i in range(nbins)])

print('Relative constraints on sigma8(z) (fixed shape, w CMB lensing):',s8_constraints_combined)

Relative constraints on sigma8(z) (fixed shape, w CMB lensing): [0.00898109 0.00683269 0.01183671]
